<div style="overflow: hidden;">
    <img src="images/DREGS_logo_v2.png" width="300" style="float: left; margin-right: 10px;">
</div>

# Getting started: Part X - Using dataregistry with GCRCatalogs

Here we show how to access catalogs belonging to the `GCRCatalogs` package via the information stored in the dataregistry.

### What we cover in this tutorial

In this tutorial we will learn how to:

1) Find and read the catalogs using the standard GCRCatalogs interface
2) Query catalog metadata directly using the data registry, then use that metadata to find and read catalogs

### Before we begin

Currently (November, 2024) the required versions of gcr-catalogs and dataregistry are only available in the `desc-python-bleed` kernel. Make sure you have selected that kernel while running this tutorial.

If you haven't done so already, check out the [getting setup](https://lsstdesc.org/dataregistry/tutorial_setup.html) page from the documentation if you want to run this tutorial interactively.

## 1) Using the usual GCRCatalogs interface

Note that, using this method, we will not be calling any data registry services directly, but the data registry database still must be accessible. That means you must have gone through at least part of the tutorial setup referred to above, in particular the steps of creating a couple small files needed for authentication. See details [here](http://lsstdesc.org/dataregistry/installation.html#one-time-setup).

### Configuring GCRCatalogs 

A quick way to check everything is set up correctly is to run the first cell below, which should load the GCRCatalogs package, and print the package version. It should be at least 1.9.0.

In [ ]:
import GCRCatalogs
print(f"Working with GCRCatalogs version: {GCRCatalogs.__version__}")

We need to tell `GCRCatalogs` whether to use the old-style metadata access method (reading config files) or to fetch metadata from the data registry.  There are two ways to do this:

1. Before running, set the environment variable `GCR_CONFIG_SOURCE`to one of the two allowed values: "files" or "dataregistry"
2. Invoke the GCRCatalogs routine `ConfigSource.set_config_source`

Here we use the second method.


In [ ]:
# Tell GCRCatalogs to use the data registry
GCRCatalogs.ConfigSource.set_config_source(dr=True)

Now we can use any of the standard GCRCatalogs query routines.

In [ ]:
# Find catalogs whose name starts with "cosmo"
cosmos = GCRCatalogs.get_available_catalog_names(name_startswith="cosmo")
cosmos

In [ ]:
# Load a catalog; find out something about it
cat = GCRCatalogs.load_catalog("cosmoDC2_v1.1.4_small")
cat.native_filter_quantities

## 2) Using the data registry directly

We learned how to connect to the DESC data registry in other tutorials using the `DataRegistry` class. Let's connect again using the defaults _except_ for the schema.  Since the catalogs maintained by GCRCatalogs are stored in the DESC production shared area, their database entries are in the production schema, not in the (default) working schema.



In [ ]:
from dataregistry import DataRegistry
from dataregistry.schema import DEFAULT_SCHEMA_PRODUCTION

# Establish connection to the production schema
datareg = DataRegistry(schema=DEFAULT_SCHEMA_PRODUCTION)

### Dataset attributes

Recall that a `DataRegistry` instance has a member `Query` which provides all the query services.

As described in "Getting started: Part 3 - Simple queries" you can ask for values of attributes of datasets, subject to one or more filters. You can find out what those attributes ("columns" in database parlance) are with one of those services:

In [ ]:
all_columns = datareg.Query.get_all_columns()
print(all_columns)

That is a list of __all__ columns from __all__ tables, maybe more than we bargained for. Let's restrict it to columns in the `dataset` table.

In [ ]:
dataset_columns = [col for col in all_columns if col.startswith('dataset.')]
print(dataset_columns)